In [6]:
from pprint import pprint
import sys 
from nltk.corpus import brown
from nltk.corpus import reuters
import nltk
from nltk.corpus import PlaintextCorpusReader
from textblob import TextBlob
import gramformer
from gramformer import Gramformer
gf = Gramformer(models=1, use_gpu=False)
from nltk.translate.bleu_score import sentence_bleu

/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:34: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/transformers/models/auto/tokenization_auto.py:720: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/transformers/models/auto/auto_factory.py:466: FutureWarning: The `use_auth_token` argument is deprecated an

[Gramformer] Grammar error correct/highlight model loaded..


/Users/kiran_g/Library/Python/3.9/lib/python/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [7]:
import requests
from bs4 import BeautifulSoup
import string
import nltk
nltk.download('punkt')  # Download sentence tokenizer
from nltk.tokenize import sent_tokenize,word_tokenize
url = "https://en.wikipedia.org/wiki/India" # URL of the website to scrape
response = requests.get(url)# Fetch the webpage content
if response.status_code == 200:# Check if the request was successful
    soup = BeautifulSoup(response.content, 'html.parser')
    paragraphs = soup.find_all('p')# Find all paragraphs
    with open("extracted_sentences.txt", "w") as file: # Open a text file to save the extracted sentences
        for paragraph in paragraphs: # Loop through all paragraphs  # Get the text content of each paragraph
            text = paragraph.get_text() # Split the text into individual sentences
            sentences = sent_tokenize(text)  # Use NLTK to split into sentences
            for sentence in sentences:
                file.write(sentence.strip() + "\n")
    print("Sentences extracted and saved to 'extracted_sentences.txt'")
else:
    print(f"Failed to fetch the webpage. Status code: {response.status_code}")
def remove_punctuation(text):
    tokens = word_tokenize(text) # Tokenize the text into words
    tokens = [word for word in tokens if word.isalnum()]# Keep only alphanumeric tokens (remove punctuation)
    cleaned_text = ' '.join(tokens)  # Join tokens back into a cleaned string
    return cleaned_text
    
# Paths to existing  text files
input_files = ["extracted_sentences.txt", "chat.txt"]  # List of input file paths
output_file_path = "combined_cleaned.txt"  # Output file path

cleaned_texts = []# Initialize a list to store cleaned texts
# Read and clean each text file
for input_file_path in input_files:
    with open(input_file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        cleaned_text = remove_punctuation(text)
        cleaned_texts.append(cleaned_text)
        
combined_cleaned_text = ' '.join(cleaned_texts)# Concatenate all cleaned texts
with open(output_file_path, 'w', encoding='utf-8') as file:# Write the combined cleaned text to the output file
    file.write(combined_cleaned_text)

print("Combined cleaned text written to:", output_file_path)# Print the final combined cleaned text or a success message


[nltk_data] Downloading package punkt to /Users/kiran_g/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Sentences extracted and saved to 'extracted_sentences.txt'
Combined cleaned text written to: combined_cleaned.txt


In [8]:
import nltk
from nltk.corpus import PlaintextCorpusReader, brown
import os


def get_trigram_freq(tokens):# Function to calculate trigram frequency distribution
    tgs = list(nltk.trigrams(tokens))    # Create a list of trigrams from the given tokens
    a, b, c = list(zip(*tgs))    # Unzip the trigrams into three separate lists
    bgs = list(zip(a, b))# Create bigrams from the first two elements of each trigram
    # Return a Conditional Frequency Distribution where the key is a bigram
    # and the frequency distribution is over the third element of each trigram
    return nltk.ConditionalFreqDist(list(zip(bgs, c)))


def get_bigram_freq(tokens):# Function to calculate bigram frequency distribution
    bgs = list(nltk.bigrams(tokens)) # Create a list of bigrams from the given tokens
    return nltk.ConditionalFreqDist(bgs)# Return a Conditional Frequency Distribution of the bigrams


def appendwithcheck(preds, to_append):# Function to append an item to a list, ensuring no duplicate keys
    for pred in preds:# Check if the first element of 'to_append' already exists in 'preds'
        if pred[0] == to_append[0]:
            return  # If it exists, do nothing and return
    preds.append(to_append)# If it doesn't exist, append 'to_append' to 'preds'


def incomplete_pred(words, n):# Function to get predictions for incomplete words based on previous context
    all_succeeding = bgs_freq[(words[n-2])].most_common() # Get the most common succeeding words for a given bigram
    preds = []# Initialize a list to store predictions and a counter
    number = 0
    for pred in all_succeeding:# Check for words that start with the same prefix as the incomplete word
        if pred[0].startswith(words[n-1]):# If the first character of 'pred' matches the incomplete word's first character
            appendwithcheck(preds, pred)  # Add it to 'preds'
            number += 1  # Increment the counter
        if number == 3: # If we have 3 predictions, return them
            return preds

    if len(preds) < 3:# If there are fewer than 3 predictions, calculate edit distances
        med = []# Create a list of tuples (word, edit distance from the incomplete word)
        for pred in all_succeeding:
            med.append((pred[0], nltk.edit_distance(pred[0], words[n-1], transpositions=True)))
        med.sort(key=lambda x: x[1]) # Sort the list by edit distance

        index = 0  # Index to iterate through 'med'
        while len(preds) < 3:# Continue until there are at least 3 predictions
            if index < len(med):  # Ensure we don't go out of bounds
                if med[index][1] > 0: # If edit distance is greater than zero, append the prediction
                    appendwithcheck(preds, med[index])
                index += 1
            if index >= len(med):# If the index exceeds the length of 'med', return the current predictions
                return preds
    
    return preds  # Return the final list of predictions

new_corpus = PlaintextCorpusReader('./', '.*')# Load a new corpus and get tokens from a specific file along with Brown corpus
tokens = brown.words() + new_corpus.words('extracted_sentences.txt')

bgs_freq = get_bigram_freq(tokens)# Generate bigram and trigram frequency distributions from the tokens
tgs_freq = get_trigram_freq(tokens)


In [9]:
import nltk
from textblob import TextBlob  # For text correction
from nltk.corpus import PlaintextCorpusReader, brown

def predict_page():# Function to predict the next word(s) given an input sentence
    enter = input("Type sentence for predicting next word ")# Get a sentence from user input
    # Split the input string into individual words
    words = enter.strip().split()  # Trim and split into words
    n = len(words)    # Get the number of words in the input
    work = "pred" # Variable to control prediction mode
    last_word = words[-1]  # Get the last word from the input
    if last_word in bgs_freq: # Determine if there's bigram frequency data for the last word
        if len(bgs_freq[last_word].most_common()) == 0: # If there are no common bigrams, treat it as incomplete prediction
            inc = incomplete_pred(words, n)# Call the incomplete prediction function with current words
            print("inc ->", inc)
            return inc  # Exit early after printing incomplete predictions
        else:
            work = "pred" # The word is recognized, continue with prediction
    else:
        inc = incomplete_pred(words, n)# If the last word is not in bigram frequency, treat it as incomplete
        print("inc ->", inc)
        return inc
    
    if work == "pred":# Prediction mode for complete words
        if n == 1:# Prediction for a single word input
            bgs = bgs_freq[(enter.strip())].most_common(5)  # Get top 5 bigrams
            pred = [i[0] for i in bgs]  # Extract the predicted words
        elif n > 1:
            tgs = tgs_freq[(words[n-2], words[n-1])].most_common(5)# Prediction for two or more words using trigrams
            pred = [i[0] for i in tgs]  # Extract predicted words
    print("Below are the top 5 predicted  words :")# Display predictions
    return pred  # Return the predicted words

def predict_page1():# Function to correct a given sentence
    enter = input("Enter sentence for correction ")# Get a sentence from user input
     # Use TextBlob to correct the sentence
    corrected_sentence = TextBlob(enter.strip()).correct()  # Correction via TextBlob
    corrected_sentence_str = str(corrected_sentence) # Convert the corrected sentence to a string
    corrections = gf.correct(corrected_sentence_str) # Perform additional corrections if needed (assuming `gf` has a `correct` function) 
    print("Below is the sentence after correction:")
    print(corrections)
    # Perform additional corrections if needed (assuming `gf` has a `correct` function)
    return enter , corrections# Return the corrected sentence


In [13]:
print(" SENTENCE CORRECTION AND WORD GENARATOR \n \n")
print("1 WORD GENARATOR ")
print("2 SENTENCE CORRECTION ")
print(" please Enter number below ")
input_ = int(input())
if input_ == 1:
    print(predict_page())
elif input_ == 2:
    sentence, Correction  = predict_page1()
else:
    print("Enter Correct input")


 SENTENCE CORRECTION AND WORD GENARATOR 
 

1 WORD GENARATOR 
2 SENTENCE CORRECTION 
 please Enter number below 


 1
Type sentence for predicting next word  this is not


Below are the top 5 predicted  words :
['a', 'the', 'to', 'so', 'only']


In [18]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction
from textblob import TextBlob

# Original sentence with errors
input_sentence = "The cafe was full of custmors but the servce was too slow."

# Correct the sentence with TextBlob
corrected_sentence = TextBlob(input_sentence).correct()  # Correcting the original sentence
corrected_sentence_str = str(corrected_sentence) # Convert the corrected sentence to a string
corrections = gf.correct(corrected_sentence_str)
corrected_tokens = []
for sentence in corrections:
    corrected_tokens.extend(sentence.split())
    
# print(type(corrections))
# # Convert corrected sentence to tokenized list
# corrected_tokens = list(corrections.words)
# Reference sentence (expected correction)
reference_sentence = "The café was full of customers, but the service was too slow."

# Tokenize the reference sentence for BLEU calculation
reference_tokens = list(TextBlob(reference_sentence).words)

# Calculate the BLEU score
smoothing = SmoothingFunction()

bleu_score = sentence_bleu(
    [reference_tokens],  # The correct sentence(s) to compare against
    corrected_tokens,  # The corrected output
    smoothing_function=smoothing.method1,  # Smoothing to prevent zero scores
    weights=(0.25, 0.25, 0.25, 0.25)  # Default BLEU weights for 1-grams to 4-grams
)

# Display the results
print("Original sentence:", input_sentence)
print("Corrected sentence:", corrected_sentence)
print("Reference sentence:", reference_sentence)
print("BLEU score:", bleu_score)


Original sentence: The cafe was full of custmors but the servce was too slow.
Corrected sentence: The cafe was full of customs but the service was too slow.
Reference sentence: The café was full of customers, but the service was too slow.
BLEU score: 0.4366835442847812
